# Aprendizem Profunda - DNN
### Tarefa III
1. Rúben Gonçalo Araújo da Silva pg57900   
2. José Luis Fraga Costa pg55970
3. Pedro Miguel Costa Azevedo pg57897
4. Rui Pedro Fernandes Madeira Pinto pg56010

# Implementação

### imports

1. pandas
2. tensorflow
3. sklearn

In [5]:
# Standard Library
import os
import csv

# Data Handling
import pandas as pd
import numpy as np

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# TensorFlow / Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Embedding, Flatten, Dense, Dropout, SimpleRNN, GRU, LSTM, 
    Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D, 
    BatchNormalization, Conv1D
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Hugging Face Transformers
from transformers import (
    AutoTokenizer, 
    TFAutoModelForSequenceClassification, 
    pipeline
)


### Input dos dados

In [6]:
df = pd.read_csv("data/small_dataset.csv")
df = df.dropna(subset=["Label"])
df["Text"] = df["Text"].fillna("")
all_ids = df["ID"].fillna("")
test_ids = df["ID"].fillna("")


### Função pra gravar em csv

In [ ]:
def to_csv(results_df, name):
    os.makedirs("submissao2", exist_ok=True)
    path = f"submissao2/{name}.csv"
    filtered_df = results_df[results_df["ID"].notna() & (results_df["ID"] != "")].copy()
    filtered_df['prefix'] = filtered_df['ID'].str.extract(r'(D[0-9])')
    filtered_df['number'] = filtered_df['ID'].str.extract(r'-(\d+)').astype(int)
    filtered_df = filtered_df.sort_values(by=['prefix', 'number'])

    with open(path, 'w') as f:
        f.write("ID\tLabel\n") 
        for index, row in filtered_df.iterrows():
            f.write(f"{row['ID']}\t{row['Label']}\n") 

In [8]:
def predict_and_save(model, X_data, ids, model_name):
    # Generate predictions
    y_pred = (model.predict(X_data) > 0.5).astype(int).flatten()

    # Convert numeric predictions to labels
    labels = ["Human" if label == 0 else "AI" for label in y_pred]
    # Create results DataFrame
    results_df = pd.DataFrame({
        "ID": ids,
        "Label": labels
    })
    to_csv(results_df, model_name)

### Train Test Split (divisão de dados)

In [9]:
# Remove duplicates by keeping the first occurrence
df_unique = df.drop_duplicates(subset=["Text"], keep="first")

# Check how many rows remain
print(f"Original dataset size: {len(df)}, Unique texts: {len(df_unique)}")

# Split the deduplicated dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_unique["Text"], df_unique["Label"], test_size=0.2, random_state=42, stratify=df_unique["Label"]
)
test_indices = test_texts.index

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

# Verify no overlap
print(f"Overlap between train and test after deduplication: {len(set(train_texts) & set(test_texts))}")


Original dataset size: 4053, Unique texts: 4051
Overlap between train and test after deduplication: 0


### Tokenizer

In [10]:
# Tokenize the data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_texts)

X_train_seq = tokenizer.texts_to_sequences(train_texts)
X_val_seq = tokenizer.texts_to_sequences(val_texts)
X_test_seq = tokenizer.texts_to_sequences(test_texts)

X_train = pad_sequences(X_train_seq, maxlen=100)
X_val = pad_sequences(X_val_seq, maxlen=100)
X_test = pad_sequences(X_test_seq, maxlen=100)

# Convert string labels to numeric values
label_map = {"Human": 0, "AI": 1}  # Define mapping
y_train = np.array([label_map[label] for label in train_labels], dtype=np.float32)
y_val = np.array([label_map[label] for label in val_labels], dtype=np.float32)
y_test = np.array([label_map[label] for label in test_labels], dtype=np.float32)

## Dataset de Avaliação

In [11]:
data_set_professor = pd.read_csv("data/dataset3_inputs.csv", sep=';', on_bad_lines='skip')
data_set_professor["Text"] = data_set_professor["Text"].fillna("")
X_professor_seq = tokenizer.texts_to_sequences(data_set_professor["Text"])
X_professor = pad_sequences(X_professor_seq, maxlen=100)  

### Early Stopping

In [12]:
early_stopping = EarlyStopping(
    monitor='val_loss',     
    patience=10,            
    restore_best_weights=True,           
)

### Reduce_lr

In [13]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5, 
    patience=3,
    min_lr=0.00005
)

## CNN

In [24]:
model_cnn = Sequential([
    Embedding(input_dim=10000, output_dim=32, input_length=100),
    Dropout(0.2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])

model_cnn.compile(
    loss='binary_crossentropy',
    optimizer=Adam(0.001),
    metrics=['accuracy']
)

model_cnn.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/30


c:\Users\ruben\Desktop\Minho\MEI\SI\AP\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6629 - loss: 0.6576 - val_accuracy: 0.9306 - val_loss: 0.2878 - learning_rate: 0.0010
Epoch 2/30
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9355 - loss: 0.2219 - val_accuracy: 0.9599 - val_loss: 0.1133 - learning_rate: 0.0010
Epoch 3/30
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9745 - loss: 0.0821 - val_accuracy: 0.9753 - val_loss: 0.0673 - learning_rate: 0.0010
Epoch 4/30
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9910 - loss: 0.0443 - val_accuracy: 0.9815 - val_loss: 0.0511 - learning_rate: 0.0010
Epoch 5/30
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9941 - loss: 0.0290 - val_accuracy: 0.9846 - val_loss: 0.0425 - learning_rate: 0.0010
Epoch 6/30
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9966 - loss: 0.0186 - val_accuracy: 0.9877 - val_loss: 0.0367 - learning_rate: 0.0010
Epoch 7/30
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9974 - loss: 0.0151 - val_accuracy: 0.9892 - va

## Simple RNN

# Correr Modelos

**Test Set**

In [22]:
test_ids = df.loc[test_indices, "ID"]  # Get IDs for test split only
predict_and_save(model_cnn, X_test, test_ids, "cnn")


26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


**Avaliação**

In [23]:
predict_and_save(model_cnn, X_professor, data_set_professor["ID"], "cnn_professor")


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


## Classification Reports

In [20]:
print(classification_report(y_test, model_cnn.predict(X_test) > 0.5))

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99       420
         1.0       0.99      0.98      0.99       391

    accuracy                           0.99       811
   macro avg       0.99      0.99      0.99       811
weighted avg       0.99      0.99      0.99       811

